In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/predicting-heart-disease-baseline-simple/__results__.html
/kaggle/input/predicting-heart-disease-baseline-simple/submission.csv
/kaggle/input/predicting-heart-disease-baseline-simple/test_preds.csv
/kaggle/input/predicting-heart-disease-baseline-simple/__notebook__.ipynb
/kaggle/input/predicting-heart-disease-baseline-simple/__output__.json
/kaggle/input/predicting-heart-disease-baseline-simple/oof_preds_xgb.csv
/kaggle/input/predicting-heart-disease-baseline-simple/custom.css
/kaggle/input/playground-series-s6e2/sample_submission.csv
/kaggle/input/playground-series-s6e2/train.csv
/kaggle/input/playground-series-s6e2/test.csv
/kaggle/input/predicting-heart-disease-logistic-regression/test_pred_lr.csv
/kaggle/input/predicting-heart-disease-logistic-regression/__results__.html
/kaggle/input/predicting-heart-disease-logistic-regression/oof_pred_lr.csv
/kaggle/input/predicting-heart-disease-logistic-regression/submission.csv
/kaggle/input/predicting-heart-disease-logistic-reg

In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s6e2/train.csv')

In [3]:
train.head()

,id,Age,Sex,Chest pain type,BP,Cholesterol,FBS over 120,EKG results,Max HR,Exercise angina,ST depression,Slope of ST,Number of vessels fluro,Thallium,Heart Disease
0,0,58,1,4,152,239,0,0,158,1,3.6,2,2,7,Presence
1,1,52,1,1,125,325,0,2,171,0,0.0,1,0,3,Absence
2,2,56,0,2,160,188,0,2,151,0,0.0,1,0,3,Absence
3,3,44,0,3,134,229,0,2,150,0,1.0,2,0,3,Absence
4,4,58,1,4,140,234,0,2,125,1,3.8,2,3,3,Presence


In [4]:
target = train['Heart Disease']

In [5]:
oof_pred_lr = pd.read_csv('/kaggle/input/predicting-heart-disease-logistic-regression/oof_pred_lr.csv')

In [6]:
oof_pred_lr.head()

,oof_pred_lr
0,0.996787
1,0.006291
2,0.011770
3,0.080762
4,0.998608


In [7]:
oof_preds_xgb = pd.read_csv('/kaggle/input/predicting-heart-disease-baseline-simple/oof_preds_xgb.csv')

In [8]:
oof_preds_xgb.head()

,oof_preds_xgb
0,0.997527
1,0.009253
2,0.011418
3,0.042925
4,0.998857


In [9]:
target

0         Presence
1          Absence
2          Absence
3          Absence
4         Presence
            ...   
629995     Absence
629996     Absence
629997    Presence
629998    Presence
629999     Absence
Name: Heart Disease, Length: 630000, dtype: object

In [10]:
target = target.map({'Presence':1,'Absence':0})

In [11]:
target

0         1
1         0
2         0
3         0
4         1
         ..
629995    0
629996    0
629997    1
629998    1
629999    0
Name: Heart Disease, Length: 630000, dtype: int64

In [12]:
train_df = pd.DataFrame({
    "pred_lr":oof_pred_lr['oof_pred_lr'],
    "pred_xgb":oof_preds_xgb['oof_preds_xgb'],
    "target":target
})

In [13]:
train_df.head()

,pred_lr,pred_xgb,target
0,0.996787,0.997527,1
1,0.006291,0.009253,0
2,0.011770,0.011418,0
3,0.080762,0.042925,0
4,0.998608,0.998857,1


In [14]:
# test oofs

test_pred_lr = pd.read_csv('/kaggle/input/predicting-heart-disease-logistic-regression/test_pred_lr.csv')
test_pred_xgb = pd.read_csv('/kaggle/input/predicting-heart-disease-baseline-simple/test_preds.csv')

In [15]:
test_pred_xgb.head()

,id,test_preds
0,630000,0.962699
1,630001,0.010248
2,630002,0.987262
3,630003,0.007216
4,630004,0.173199


In [16]:
test_pred_lr.head()

,id,test_pred_lr
0,630000,0.966748
1,630001,0.003056
2,630002,0.994034
3,630003,0.009058
4,630004,0.115813


In [17]:
test_ids = test_pred_lr['id']

In [18]:
test_df = pd.DataFrame({
    'pred_lr':test_pred_lr['test_pred_lr'],
    'pred_xgb':test_pred_xgb['test_preds']
})

In [19]:
test_df.head()

,pred_lr,pred_xgb
0,0.966748,0.962699
1,0.003056,0.010248
2,0.994034,0.987262
3,0.009058,0.007216
4,0.115813,0.173199


In [20]:
test_df.shape

(270000, 2)

In [21]:
#applying ridge as meta model

In [22]:
X = train_df.iloc[:,:2]

In [23]:
y = train_df['target']

In [24]:
X.head()

,pred_lr,pred_xgb
0,0.996787,0.997527
1,0.006291,0.009253
2,0.011770,0.011418
3,0.080762,0.042925
4,0.998608,0.998857


In [25]:
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

In [26]:
from scipy.special import expit

In [27]:
N_SPLITS = 5
skf = StratifiedKFold(n_splits = N_SPLITS, shuffle= True, random_state=42)

oof_pred_meta = np.zeros(len(X))
pred_test_meta = np.zeros(len(test_df))

for train_idx,val_idx in skf.split(X,y):
    X_train,X_val = X.iloc[train_idx].copy(),X.iloc[val_idx].copy()
    y_train,y_val = y.iloc[train_idx].copy(),y.iloc[val_idx].copy()

    model = RidgeClassifier(alpha=1.0)
    model.fit(X_train,y_train)

    val_pred = expit(model.decision_function(X_val))
    oof_pred_meta[val_idx] = val_pred

    fold_cv = roc_auc_score(y_val,val_pred)
    print(f'fold_cv:{fold_cv}') 
    
    pred_test_meta += expit(model.decision_function(test_df)) / N_SPLITS

final_cv = roc_auc_score(y,oof_pred_meta)
print(f'final_cv: {final_cv}') 

fold_cv:0.9553952899053765
fold_cv:0.9544452431154287
fold_cv:0.9552869563766828
fold_cv:0.9548049589404464
fold_cv:0.9556179224273784
final_cv: 0.9551061087084185


In [28]:
sample = pd.read_csv('/kaggle/input/playground-series-s6e2/sample_submission.csv')

In [29]:
sample.head()

,id,Heart Disease
0,630000,0
1,630001,0
2,630002,0
3,630003,0
4,630004,0


In [30]:
submission = pd.DataFrame({
    "id":test_ids,
    "Heart Disease":pred_test_meta
})

In [31]:
submission.to_csv('submission.csv',index=False)